### Initial Comment

This script allows to access the MiniPreço website and retrieve the data that is stored on the website specific tags.

Using the requests and the bs4 libraries was accessed the html code of the different webpages and retrieved the information regarding the name and the price of the products that were listed there.

The data that was obtained via this method was then transformed into a string and manipulated to get the product name and price on a readable format.

Based on the excel file _Categories.xlsx_, that was inputed, it was defined that the code would loop through the names of the products and would search for that name on the products webpage of the MiniPreço website.
 - the group opted to create a new url based on the "base url" of the webpage and replacing the words that had the search criteria for the ones on this file.

When accesing a url the data was stored on a data frame. This df will then be filtered using the function data_cleaning, which removes all searches that are out of scope for that search.
 - for example, when searching meat products is usual to have animal food on the search results, which is out of the scope of the search, therefore these items would be removed
Based on the removed results then the products would be narrowed for the top 10 products, that were selected based on the search preferences that each supermarket marked as more relevant.

These individual product dataframes were then concatenated into one dataframe that stores the information of this specific supermarket.

To this dataframe are then created new columns that have some summary statistics of the data on each row, detailing the minimum price, maximum and average price for each row, as well as the number of products that were passed onto the final dataframe for each product.

These data are to be then used on the next steps of the project

In [ ]:
import requests
from bs4 import BeautifulSoup
import re # For string transformation
import pandas as pd
import unidecode

In [2]:
# Read the initial excel file with all product names to be searched on and all the string expections
data = pd.read_excel("Categories.xlsx")
data["Product"] = data["Product"].replace('Vaca', 'Bovino')
data_subset = data.iloc[:,:1]

## String transformation to remove all uppercase and all accents (text normalisation)
data_subset = data_subset.applymap(lambda row: unidecode.unidecode(row).lower() if type(row) == str else row)
data_subset.head()

,Product
0,peru
1,frango
2,bacalhau
3,cebola
4,batata


### Explanation Comment
The below function retrieves the data based of the Continente's product's web-page.

Based on the information on the web-page it will create a dataframe with all the Product Names and respective Product Prices on that page.

In [3]:
def get_name_price(url):
    
    #1. Get web data into the notebook
    page = requests.get(url)
    if page.status_code == 200:
        pass
    elif page.status_code == 404:
        print('This page was not found.')
        return 10
    
    soup = BeautifulSoup(page.text, 'html.parser')
    

    
    number_products_price = len(soup.find_all('p', class_ = 'pricePerKilogram'))
    
    product_prices = soup.find_all('p', class_ = 'pricePerKilogram') # get price based on MiniPreço's website design
    product_names = soup.find_all('span', class_ = 'details') # get name based on MiniPreço's website design
    
    number_products_price = len(product_prices)
    number_products_name = len(product_names)
    
    if number_products_price != number_products_price:
        # The number of product names is different from the number of product prices
        print("The number of prices and names on this webpage is different, please check this page manually")
        # return

    product_name_list = []
    product_price_list = []
    
    for i in range(0, number_products_price):
        product_name = str(product_names[i])
        product_price = str(product_prices[i])
        
    
        # 2. String formatting
        # 2.1. Transformation of Product Name:
        #product_name = re.sub(r'[A-Z]', '',  product_name )
        product_name = re.sub('<span class="details">', '', product_name)
        product_name = re.sub('\n\t\t\t\t\t', '', product_name)
        product_name = re.sub(r'[0-9]', '', product_name)
        product_name = re.sub('g</sp', '', product_name)
        product_name = re.sub('g</span>', '', product_name)
        product_name = re.sub('an>', '', product_name)
        product_name = re.sub('</span>', '', product_name)
        product_name = re.sub('aprox', '', product_name)
        product_name = re.sub('</sp', '', product_name)
        product_name = re.sub(r'\([^)]*\)', '', product_name)
        #product_name = product_name[12:]
        
        
        
        # 2.2. Transformation of Product Price
        product_price = re.sub('<p class="pricePerKilogram">', '', product_price)
        #product_price = re.sub('(', '', product_price)
        #product_price = re.sub(')', '', product_price)
        product_price = re.sub('/Kg.', '', product_price)
        product_price = re.sub('€', '', product_price)
        product_price = re.sub('</p>', '', product_price)
        product_price = re.sub('\n<s>', '', product_price)
        product_price = re.sub('</s>\n\t\t\t\t\t\t\t\t\t', '', product_price)
        product_price = re.sub('\n\t\t\t\t\t\t\t\t', '', product_price)
        product_price = re.sub(r'[()]', '', product_price)
        product_price = product_price[0: 4]
        
        
        # 3. Append product name and price to the correspondent lists
        product_name_list.append(product_name)
        product_price_list.append(product_price)
        
    # 4. Create a dataframe to store all the data
    output_df = pd.DataFrame(list(zip(product_name_list,product_price_list)), columns = ["Product Name", "Product Price"])
    return output_df

### Explanation Comment
The below **_function_** allows the definition of the data cleaning process based on the "forbiden strings" for each product type
 - Please be aware that the column position on the original dataframe that has the information regarding the forbiden strings is hard coded and at the moment has the column number = 2. If the original data set changes then we need to adapt this position

This function sets up the top 10 products for each search object, and removes from it some user defined strings that may confound the products to be analysed.
 - i.e. if we search for "peru" we may have some results regarding animal food that has turkey, and also some other meat products made from turkey. These can be defined initially to avoid including these data on the final dataframe

In [4]:
def data_cleaning(url_data, index, original_data = data):
    
    strings_to_remove = original_data.iloc[index, 2] # get the set of strings to remove
    
    if pd.isna(strings_to_remove) == True:
    # Meaning that the forbiden strings field is empty
        strings_to_remove = ""
    
    data_10 = url_data[url_data["Product Name"].str.contains(strings_to_remove,)==False] # Include the specific strings that may confound the search for this product
    
    data_10 = data_10.iloc[:10,:] # Search for the top 10 products sorted by relevance and from the main brand7

    return data_10

### Explanation Comment
The below loop will iterate across all products that we've defined in the initial excel.

The **search_url** object has a fixed structure that will define the website url to be search on.
 - Using this url the function **get_name_price** will be called to retrieve the data from the web using the built url
 - The data will then be cleaned to have the top10 products on each search term that fits the description

The output of each iteration will be a dataframe with all relevant product names and prices for each product that is being searched

In [5]:
# For tests, when deploying please remove the if condition so that it can run for all product that are placed on the initial excel


for index, row in data_subset.iterrows():
    if index == 0:
        search_object = row["Product"]
        #search_url = str("https://lojaonline.minipreco.pt/search?q="+str(search_object)+"%3Arelevance%3Abio%3Atrue&text="+str(search_object)+"#")
        
        search_url = str("https://www.minipreco.pt/search?q=" + 
                         str(search_object) + 
                         "%3Arelevance%3Abio%3Atrue&text=" + 
                         str(search_object) + "#")
        
        #search_url = str("https://lojaonline.minipreco.pt/search?q=frango%3Arelevance%3Abio%3Atrue&text=frango#")
        strings_to_remove = data.iloc[index, 2] # get the set of strings to remove
  
        ## Call the function to retrieve the data from the web
        all_data = get_name_price(search_url) # tirado do site
        
        all_data["Supermarket"] = "Mini Preço"
        all_data["Category Object"] = data.iloc[index, 1] # Adding
        all_data["Product Object"] = data.iloc[index, 0] #
        
        

        # Clean the results obtained so that we can have only the results that are of interest to us
        product_data = data_cleaning(url_data = all_data, index = index, original_data = data)
        aggregate_data = product_data # pass the data to the aggregate data frame
    
    else:
    #elif index == 1:
        
        search_object = row["Product"]
        search_url = str("https://www.minipreco.pt/search?q=" + 
                         str(search_object) + 
                         "%3Arelevance%3Abio%3Atrue&text=" + 
                         str(search_object) + "#")
        
        strings_to_remove = data.iloc[index, 2] # get the set of strings to remove
  
        ## Call the function to retrieve the data from the web
        all_data = get_name_price(search_url)
        
        all_data["Supermarket"] = "Mini Preço"
        all_data["Category Object"] = data.iloc[index, 1] # Adding Category to the dataframe
        all_data["Product Object"] = data.iloc[index, 0] # Adding Product to the dataframe
        
        
        # Clean the results obtained so that we can have only the results that are of interest to us
        product_data = data_cleaning(url_data = all_data, index = index, original_data = data)
        
        aggregate_data = pd.concat([aggregate_data, product_data])

    
    #else:
     #   pass

In [6]:
aggregate_data

,Product Name,Product Price,Supermarket,Category Object,Product Object
4,CORTE DO DIA Bifes Peru Embalado,"7,79",Mini Preço,Talho,Perú
5,CORTE DO DIA Espetadas de Peru Embaladas,"6,49",Mini Preço,Talho,Perú
10,CORTE DO DIA Pernas de Frango Embaladas,"4,45",Mini Preço,Talho,Perú
11,CORTE DO DIA Perninhas de Frango Embaladas,"5,44",Mini Preço,Talho,Perú
2,DIA SIMPRATICO Hambúrguer de Frango Sem Glúten,"8,31",Mini Preço,Talho,Frango
...,...,...,...,...,...
5,DIA FIDIAS Iogurte Grego Baunilha x,"3,38",Mini Preço,Laticinios,Iogurtes
6,DIA FIDIAS Iogurte Grego Caramelo x,"3,38",Mini Preço,Laticinios,Iogurtes
7,DIA LÁCTEA Iogurte Líquido Stracciatella x,"2,45",Mini Preço,Laticinios,Iogurtes
8,DIA LÁCTEA Iogurte Líquido Aroma Banana,"2,25",Mini Preço,Laticinios,Iogurtes


In [7]:
aggregate_data["Product Price"]=aggregate_data["Product Price"].replace(',', '.', regex=True)

In [8]:
aggregate_data["Product Price"] = aggregate_data["Product Price"].astype(float)

In [9]:
print(aggregate_data.dtypes)

Product Name        object
Product Price      float64
Supermarket         object
Category Object     object
Product Object      object
dtype: object


In [10]:
final_df = aggregate_data.groupby(['Supermarket','Category Object','Product Object']).agg({'Product Price': [("Average Per Product", "mean"),("Min Price per Product", "min"),("Max Price per Product", "max") ,("Count of Products", "count")]})
final_df

Product Price  \
                                             Average Per Product   
Supermarket Category Object Product Object                         
Mini Preço  Bebidas         Agua Garrafão               0.340000   
            Casa            Detergente Loiça            1.358000   
                            Papel higienico             0.321667   
                            Sacos do Lixo               0.252500   
            Charcutaria     Fiambre                    11.530000   
            Congelados      Ervilhas                    1.120000   
            Enlatados       Grão                        1.285000   
            Higiene         Sabonete                    2.240000   
            Laticinios      Iogurtes                    2.620000   
                            Leite                       3.494444   
                            Ovos                        1.662000   
                            Queijo                      7.086000   
            Legumes         Abóbora                     3.200000   
                            Alho Frances                3.230000   
                            Batata                      0.990000   
                            Cebola                      1.335000   
                            Cenoura                     0.840000   
            Mercearia       Arroz                       1.457143   
                            Azeite                      5.450000   
                            Açúcar                      1.515000   
                            Cafe                        8.000000   
                            Cereais                     4.330000   
                            Esparguete                  2.370000   
                            Farinha                     0.960000   
                            Vinagre                     2.385000   
            Padaria         Pão                         3.467000   
            Peixaria        Bacalhau                   10.790000   
            Talho           Bovino                      7.392000   
                            Frango                      5.204444   
                            Perú                        6.042500   
                            Porco                       6.637500   

                                                                    \
                                             Min Price per Product   
Supermarket Category Object Product Object                           
Mini Preço  Bebidas         Agua Garrafão                     0.14   
            Casa            Detergente Loiça                  0.90   
                            Papel higienico                   0.19   
                            Sacos do Lixo                     0.06   
            Charcutaria     Fiambre                           6.64   
            Congelados      Ervilhas                          1.12   
            Enlatados       Grão                              1.20   
            Higiene         Sabonete                          1.98   
            Laticinios      Iogurtes                          1.49   
                            Leite                             0.59   
                            Ovos                              1.31   
                            Queijo                            5.07   
            Legumes         Abóbora                           3.20   
                            Alho Frances                      3.23   
                            Batata                            0.99   
                            Cebola                            0.99   
                            Cenoura                           0.84   
            Mercearia       Arroz                             1.07   
                            Azeite                            3.80   
                            Açúcar                            0.79   
                            Cafe                              8.00   
                            Cereais                       

In [11]:
final_df.to_excel('MiniPreco_Products.xlsx')